In [34]:
import tensorflow as tf
import numpy as np
xs_face = np.load("saved_data/train_face.npy")
xs_pose = np.load("saved_data/train_pose.npy")
ys = np.load("saved_data/train_y.npy")
dataset = tf.data.Dataset.from_tensor_slices((xs_face, xs_pose, ys))
for element in dataset.as_numpy_iterator():
    print(element[0].shape)
    print(element[1].shape)
    print(element[2].shape)
    break

(7, 7, 256)
(22, 11)
(2,)


In [36]:
from tensorflow import keras
# Face.
faceInput = keras.layers.Input(shape=(7,7,256,))
flattenFace = keras.layers.Flatten()(faceInput)
denseFace = keras.layers.Dense(64, activation='relu')(flattenFace)

# Pose.
poseInput = keras.layers.Input(shape=(22, 11, ))
flattenPose = keras.layers.Flatten()(poseInput)
densePose = keras.layers.Dense(64, activation='relu')(flattenPose)

# Concat feature and pass to output layer.
merged = keras.layers.Concatenate(axis=1)([denseFace, densePose])
output = keras.layers.Dense(2, activation='softmax', use_bias=False)(merged)
model = keras.models.Model(inputs=[faceInput, poseInput], outputs=output)
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 7, 7, 256)]  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 22, 11)]     0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 12544)        0           input_5[0][0]                    
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 242)          0           input_6[0][0]                    
_______________________________________________________________________________________

In [37]:
keras.backend.clear_session()
model.fit([xs_face, xs_pose], ys, batch_size=32, epochs=100)

Epoch 1/100
22/22 [==============================] - 0s 3ms/step - loss: 0.9512 - accuracy: 0.5301
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5830
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6241
Epoch 4/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6372 - accuracy: 0.6358
Epoch 5/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6216 - accuracy: 0.6608
Epoch 6/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6654 - accuracy: 0.6226
Epoch 7/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6050 - accuracy: 0.6549
Epoch 8/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6158 - accuracy: 0.6579
Epoch 9/100
22/22 [==============================] - 0s 3ms/step - loss: 0.5886 - accuracy: 0.7078
Epoch 10/100
22/22 [==============================] - 0s 3ms/step - loss: 0.5565 - accuracy: 0.7269
Epoch 11/

In [ ]:
model.save("./saved_models/saved_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_models/saved_model/assets


In [5]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "./saved_models/tfjs_2")

In [32]:
from tensorflow.keras import layers
from tensorflow import keras
model_seq = keras.Sequential(
    [
        layers.Dense(64, activation="relu", name="hidden1", input_shape=(12786,)),
        layers.Dense(32, activation="relu", name="hidden2"),
        layers.Dense(2, activation="softmax", name="output", use_bias=False)
    ]
)
model_seq.summary()
model_seq.compile(loss='categorical_crossentropy', 
                  optimizer=keras.optimizers.Adam(1e-2),
                  metrics=['accuracy'])
model_seq.fit(np.concatenate([xs_face.reshape(xs_face.shape[0], -1), xs_pose.reshape(xs_pose.shape[0], -1)], axis=1), ys, batch_size=32, epochs=100)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 64)                818368    
_________________________________________________________________
output (Dense)               (None, 2)                 128       
Total params: 818,496
Trainable params: 818,496
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
22/22 [==============================] - 0s 3ms/step - loss: 4.0260 - accuracy: 0.5257
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5037
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5037
Epoch 4/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5037
Epoch 5/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5037
Epoch 6/100
22

In [20]:
xs_face.shape
xs_face.reshape(xs_face.shape[0], -1).shape
xs_pose.reshape(xs_pose.shape[0], -1).shape

(681, 242)

In [29]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model_seq, "./saved_models/tfjs_seq")